In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from io import StringIO
import csv
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import sqlite3

In [3]:
company_list = ['AMZN', 'TSLA', 'AAPL']
stockdata = []
def getData(company_stock):
    url = f'https://query1.finance.yahoo.com/v7/finance/download/{company_stock}?'
    params = {
           'range': '1y',
           'interval':'1mo',
           'events':'history',
           'includeAdjustedClose':'true'}
    url_info = requests.get(url,params = params, headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})
    #print(url_info.text)
    file = StringIO(url_info.text)
    reader = csv.reader(file)
    data = list(reader)
    #print(data)

    data_frame = pd.DataFrame(data[1:], columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
    data_frame.insert(1, 'Company Name', f'{company_stock}')
    return pd.DataFrame(data_frame)
    
for company in company_list:
    stockdata.append(getData(company))
    print(f'Getting: {company}')


Getting: AMZN
Getting: TSLA
Getting: AAPL


In [ ]:
combined_list = pd.concat(stockdata)
print(combined_list)


In [ ]:
closing_price = combined_list[['Date', 'Company Name', 'Close']]
print(closing_price)

In [40]:
#Using SQLite3: 
connection = sqlite3.connect('practice_finance.db')
cursor = connection.cursor()
#Creating table:
tables_1 = 'Create Table stock_data (id integer, date datetime, company_name varchar(100) , open integer, high integer, low integer, close integer, adj_close integer, volume integer)'  
cursor.execute(tables_1)

connection.commit()

In [44]:
#inserting information:
insert_amazon = 'Insert into stock_data (id, date, company_name, open, high, low, close, adj_close, volume) values (?, ?, ?, ?, ?, ?, ?, ?, ?)'
cursor = connection.cursor()
for info in combined_list.itertuples():
    cursor.execute(insert_amazon, info)
connection.commit()


In [ ]:
#Query:
query_1 = ''' 
select * from stock_data where company_name = 'AMZN'
'''
for i in cursor.execute(query_1):
    print(i)

In [21]:
query_2 = ''' 
select date, company_name, max(close), min(close) from stock_data
'''
for i in cursor.execute(query_2):
    print(i)

('2022-06-01', 'AMZN', 1144.76001, 106.209999)


In [45]:
query_3 = ''' 
select date, avg(open), avg(close), max(close), min(close) from stock_data where company_name = 'AMZN'
'''
for i in cursor.execute(query_3):
    print(i)

('2022-06-01', 154.56730823076924, 149.3275393846154, 175.3535, 106.209999)
